In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pathlib
import warnings
warnings.filterwarnings('ignore')

# Import necessary Keras layers and modules
import keras

# Import TensorFlow
import tensorflow as tf

In [ ]:
# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) `tensorflow` random seed
# 3) `python` random seed
keras.utils.set_random_seed(812)

# This will make TensorFlow ops as deterministic as possible, but it will
# affect the overall performance, so it's not enabled by default.
# `enable_op_determinism()` is introduced in TensorFlow 2.9.
tf.config.experimental.enable_op_determinism()

In [ ]:
# Read in the training split file
train_list = pd.read_csv('../data/train/train_recordings.csv', index_col=0)
# train_list.head()

# Read in the validation split file
val_list = pd.read_csv('../data/val/validation_recordings.csv', index_col=0)
# val_list.head()

# Read in the test split file
test_list = pd.read_csv('../data/val/test_recordings.csv', index_col=0)
# test_list.head()

In [ ]:
# Verify that there is no common file between the 3 splits
train_set, val_set, test_set = set(train_list['recording_id']), set(val_list['recording_id']), set(test_list['recording_id'])

In [ ]:
from scipy.io import wavfile
from scipy.signal import spectrogram
from scipy.signal import resample

# Convert the train, val and test split recordings into images
labels_map = {0: 'normal', 1: 'abnormal'}
splits = [train_list, val_list, test_list]
split_names = ['train', 'val', 'test']
for split, split_name in zip(splits, split_names):
    pathlib.Path(f'../data/{split_name}/images/normal').mkdir(parents=True, exist_ok=True)
    pathlib.Path(f'../data/{split_name}/images/abnormal').mkdir(parents=True, exist_ok=True)
    for id, label in zip(split['recording_id'], split['outcome']):
        audio_file = f'../data/preprocessed_recordings_10s/{id}.wav'
        if os.path.exists(audio_file):

            # load the .wav file
            sample_rate, signal_data = wavfile.read(audio_file)

            # downsample to 1000 Hz
            resampled_signal = resample(signal_data, int(len(signal_data) * (1000/sample_rate)))

            # Calculate spectrogram using Tukey window
            f, t, spec = spectrogram(resampled_signal, fs=1000, window='tukey', nperseg=128, noverlap=64)

            # Discard the first frequency bin
            spec = spec[1:, :]

            # Convert spectrogram values to logarithmic scale
            log_spec = np.log10(spec)

            # Normalize between 0 and 1 using min-max normalization
            Sxx = (log_spec - log_spec.min()) / (log_spec.max() - log_spec.min())

            # truncate or pad spectrogram to desired dimensions
            n_time_bins = 64
            n_freq_bins = 128

            if Sxx.shape[1] < n_freq_bins:
                padding = np.zeros((Sxx.shape[0], n_freq_bins - Sxx.shape[1]))
                Sxx = np.hstack((Sxx, padding))
            else:
                Sxx = Sxx[:, :n_freq_bins]

            if Sxx.shape[1] < n_time_bins:
                padding = np.zeros((n_time_bins - Sxx.shape[1], Sxx.shape[1]))
                Sxx = np.vstack((Sxx, padding))
            else:
                Sxx = Sxx[:n_time_bins, :]

            # reshape spectrogram to desired dimensions
            Sxx = np.reshape(Sxx, (n_time_bins, n_freq_bins, 1))

            # Plot the spectrogram
            plt.figure(figsize=(10, 5));
            plt.imshow(Sxx, cmap='jet', aspect='auto', origin='lower',
                        extent=[t.min(), t.max(), f.min(), f.max()]);

            plt.axis('off');
            plt.savefig(f'../data/{split_name}/images/{labels_map[label]}/{id}.png');
            plt.clf();


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create an image data generator for augmenting and preprocessing training images
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

# Generate batches of training data from the training set directory
training_set = train_datagen.flow_from_directory(
        '../data/train/images/',
        target_size=(256, 256),
        batch_size=32,
        classes=['normal'],
        color_mode='grayscale')

# Check the image shape and class indices
print(training_set.image_shape, training_set.class_indices)

Found 1328 images belonging to 1 classes.
(256, 256, 1) {'normal': 0}


In [ ]:
# Create an image data generator for augmenting and preprocessing validation images
val_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

# Generate batches of validation data from the validation set directory
validation_set = val_datagen.flow_from_directory(
        '../data/val/images/',
        target_size=(256, 256),
        batch_size=32,
        classes=['normal'],
        color_mode='grayscale')

print(validation_set.image_shape, validation_set.class_indices)

Found 449 images belonging to 1 classes.
(256, 256, 1) {'normal': 0}


In [ ]:
#Calculate the number of samples in the training and validation sets, and the batch size of the data generator
num_samples = len(training_set.filenames)
batch_size = training_set.batch_size
steps_per_epoch = num_samples // batch_size

#Initialize an array to store the training images as numpy arrays
x_train = np.zeros((num_samples, 256, 256, 1), dtype=np.float32)

#Loop over the batches generated by the training data generator
for i in range(steps_per_epoch):
    batch = next(training_set)
    x_batch = batch[0]
    x_train[i*batch_size:(i+1)*batch_size] = x_batch

#Calculate the number of samples in the validation set and the batch size of the data generator
num_samples = len(validation_set.filenames)
batch_size = validation_set.batch_size
steps_per_epoch = num_samples // batch_size

#Initialize an array to store the validation images as numpy arrays
x_val = np.zeros((num_samples, 256, 256, 1), dtype=np.float32)

#Loop over the batches
for i in range(steps_per_epoch):
    batch = next(validation_set)
    x_batch = batch[0]
    x_val[i*batch_size:(i+1)*batch_size] = x_batch

# Normalize pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.
x_test = x_val.astype('float32') / 255.

#Reshape the input images to have a single grayscale channel
x_train = np.reshape(x_train, (len(x_train), 256, 256, 1))
x_val = np.reshape(x_val, (len(x_val), 256, 256, 1))

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# Build the autoencoder model
input_img = Input(shape=(256, 256, 1))

# Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu',padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define the autoencoder model with input_img as the input and decoded as the output
autoencoder = Model(input_img, decoded)

# Compile the autoencoder model with adam optimizer and mean-squared error loss function
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=32,
                shuffle=True,
                validation_data=(x_val, x_val))

Epoch 1/10
42/42 [==============================] - 149s 4s/step - loss: 0.1008 - val_loss: 0.7149
Epoch 2/10
42/42 [==============================] - 133s 3s/step - loss: 5.4223e-05 - val_loss: 0.7150
Epoch 3/10
42/42 [==============================] - 133s 3s/step - loss: 1.8452e-05 - val_loss: 0.7150
Epoch 4/10
42/42 [==============================] - 132s 3s/step - loss: 1.4308e-05 - val_loss: 0.7150
Epoch 5/10
42/42 [==============================] - 152s 4s/step - loss: 1.2522e-05 - val_loss: 0.7150
Epoch 6/10
42/42 [==============================] - 133s 3s/step - loss: 1.1759e-05 - val_loss: 0.7150
Epoch 7/10
42/42 [==============================] - 143s 3s/step - loss: 1.1396e-05 - val_loss: 0.7150
Epoch 8/10
42/42 [==============================] - 135s 3s/step - loss: 1.1204e-05 - val_loss: 0.7150
Epoch 9/10
42/42 [==============================] - 134s 3s/step - loss: 1.1091e-05 - val_loss: 0.7150
Epoch 10/10
42/42 [==============================] - 134s 3s/step - loss: 1.1

In [ ]:
# Save the model weights
autoencoder.save_weights('../models/autoencoder_weights.h5')

In [ ]:
"""
Run this cell if model needs to loaded from a h5 file
"""
# from keras.models import load_model
# model = create_model()
# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# model.load_weights('../models/autoencoder_weights.h5')

In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd

# Define the folder containing the images
folder_path = '../data/test/images/abnormal'

# Initialize lists to store the recording IDs and reconstruction errors
recording_ids = []
reconstruction_errors = []

# Iterate through the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.png'):  # Check if it's an image file
        # Load and preprocess the image
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        img = img.convert('L')  # Convert to grayscale
        img = img.resize((256, 256))
        x = np.asarray(img, dtype=np.float32) / 255.0
        x = np.expand_dims(x, axis=0)  # Add batch dimension
        x = np.expand_dims(x, axis=-1)  # Add channel dimension

        # Predict the reconstruction error
        reconstructed = autoencoder.predict(x)
        error = np.mean(np.square(x - reconstructed))

        # Append recording ID and error to lists
        recording_ids.append(filename)
        reconstruction_errors.append(error)

# Create a DataFrame from the lists
data = {'Recording ID': recording_ids, 'Reconstruction Error': reconstruction_errors}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
#csv_filename = 'reconstruction_errors_abnormal.csv'
#df.to_csv(csv_filename, index=False)

print(f"Saved the reconstruction errors to {csv_filename}")

1/1 [==============================] - 0s 60ms/step
Saved the reconstruction errors to reconstruction_errors_abnormal.csv


In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd

# Define the folder containing the images
folder_path = '../data/test/images/normal'

# Initialize lists to store the recording IDs and reconstruction errors
recording_ids = []
reconstruction_errors = []

# Iterate through the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.png'):  # Check if it's an image file
        # Load and preprocess the image
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        img = img.convert('L')  # Convert to grayscale
        img = img.resize((256, 256))
        x = np.asarray(img, dtype=np.float32) / 255.0
        x = np.expand_dims(x, axis=0)  # Add batch dimension
        x = np.expand_dims(x, axis=-1)  # Add channel dimension

        # Predict the reconstruction error
        reconstructed = autoencoder.predict(x)
        error = np.mean(np.square(x - reconstructed))

        # Append recording ID and error to lists
        recording_ids.append(filename)
        reconstruction_errors.append(error)

# Create a DataFrame from the lists
data = {'Recording ID': recording_ids, 'Reconstruction Error': reconstruction_errors}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
#csv_filename = 'reconstruction_errors_normal.csv'
#df.to_csv(csv_filename, index=False)

print(f"Saved the reconstruction errors to {csv_filename}")

1/1 [==============================] - 0s 59ms/step
Saved the reconstruction errors to reconstruction_errors_normal.csv
